# Index Access of dict.keys(), dict.values(), dict.items()

`dict.keys()`, `dict.values()`, `dict.items()` don't support index access:

In [1]:
d = {"a":1, "b":2, "c":3}
k = d.keys()
k[1]

TypeError: 'dict_keys' object is not subscriptable

We need to convert them to list first or use `itertools.islice`:

In [2]:
from itertools import islice
next(islice(k, 1, None))

'b'

Here is the patch code that add index access:

In [3]:
import cffi
import ctypes
from ctypes import CFUNCTYPE, c_ssize_t, c_void_p
import sys

ffi = cffi.FFI()
ctypes.pythonapi.Py_IncRef.argtypes = [ctypes.c_size_t]

def sq_item_func(obj_addr, index):
    obj = ctypes.cast(obj_addr, ctypes.py_object).value
    if index < 0:
        index += len(dv)
    try:
        ret_obj = next(islice(obj, index, None))
        ctypes.pythonapi.Py_IncRef(id(ret_obj))
        return id(ret_obj)
    except StopIteration:
        raise IndexError("Out of range")

def patch_dict_index_access():        
    SQ_FUNC = CFUNCTYPE(c_ssize_t, c_ssize_t, c_ssize_t)
    c_sq_item_func = SQ_FUNC(sq_item_func)

    tp_as_sequence_offset = 104
    sq_item_offset = 24
    if sys.maxsize == 0x7fffffff: # if 32bit 
        tp_as_sequence_offset //= 2
        sq_item_offset //=2
        
    ffi = cffi.FFI()
    dummy_dict = {}
    to_patch_types = [type(getattr(dummy_dict, name)()) for name in ('keys', 'values', 'items')]

    for type_addr in to_patch_types:
        tp_as_sequence_addr = ffi.cast("size_t *", id(type_addr) + tp_as_sequence_offset)
        sq_item_addr = tp_as_sequence_addr[0] + sq_item_offset
        sq_item = ffi.cast('size_t *', sq_item_addr)
        sq_item[0] = ctypes.cast(c_sq_item_func, c_void_p).value
        
patch_dict_index_access()

In [4]:
k[1]

'b'

In [5]:
d.values()[-1]

3

In [6]:
d.items()[0]

('a', 1)

## How it works?

We need to set the `sq_item` field of the `tp_as_sequence` field of the type object.

```c
typedef struct _typeobject {
    PyObject_VAR_HEAD
    ...
    PySequenceMethods *tp_as_sequence; //<- here is all the sequence methods
    ...
} PyTypeObject;
```

```c
typedef struct {
    lenfunc sq_length;
    binaryfunc sq_concat;
    ssizeargfunc sq_repeat;
    ssizeargfunc sq_item; //<- this is the function corresponding to [] operator
    void *was_sq_slice;
    ssizeobjargproc sq_ass_item;
    void *was_sq_ass_slice;
    objobjproc sq_contains;

    binaryfunc sq_inplace_concat;
    ssizeargfunc sq_inplace_repeat;
} PySequenceMethods;
```

We can use a dummy dict object to get the type of the return object of `.keys()`, `.values()`, `.items()` method.